In [37]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import pickle
from utils import flat, trace
from tqdm import tqdm
import psutil
import torch
import gc
from itertools import product
from multiprocessing import Pool
from tqdm import tqdm
from data import RFCXDataset
from model import RFCXModel, kaggle_metric
from utils import sigmoid, CFG, frommelscale, tomelscale
import torch.nn.functional as F
import timm
import torch.nn as nn
from scipy.stats import rankdata

In [2]:
data_dir = '../../input/kaggle/train'
fnames = os.listdir(data_dir)
recording_ids = [x[:-5] for x in fnames]
print(len(fnames), len(os.listdir('../../input/kaggle/test')))
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
folds = joblib.load('folds.jl')
print(train_tp.shape)
display(train_tp.head())
print(train_fp.shape)
display(train_fp.head())

4729 1992
(1216, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
003bec244,14,1,44.5440,2531.250,45.1307,5531.25
006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
009b760e6,10,1,50.0854,947.461,52.5293,10852.70


(7781, 6)


,species_id,songtype_id,t_min,f_min,t_max,f_max
recording_id,,,,,,
00204008d,21,1,13.8400,3281.2500,14.9333,4125.0000
00204008d,8,1,24.4960,3750.0000,28.6187,5531.2500
00204008d,4,1,15.0027,2343.7500,16.8587,4218.7500
003b04435,22,1,43.2533,10687.5000,44.8587,13687.5000
003b04435,23,1,9.1254,7235.1562,15.2091,11283.3984


In [33]:
true = np.array([[1,1,1,0,0,0]])
pred1 = np.array([[10,9,8,2,1,0]])
pred2 = np.array([[1, 0.9, 0.8, 0.2, 0.1, 0]])
kaggle_metric(pred, true)

1.0

In [34]:
one_pred1 = pred1[0]
(one_pred1[:, np.newaxis]-one_pred1).mean(axis=1)

array([ 5.,  4.,  3., -3., -4., -5.])

In [35]:
one_pred2 = pred2[0]
(one_pred2[:, np.newaxis]-one_pred2).mean(axis=1)

array([ 0.5,  0.4,  0.3, -0.3, -0.4, -0.5])

In [36]:
pred1+pred2

array([[11. ,  9.9,  8.8,  2.2,  1.1,  0. ]])

In [16]:
tmp = torch.load('../../model/v12/stage0-fold0-seed0/best.pth', map_location='cpu')
sigmoid(tmp['val_preds']).mean(axis=0)

array([0.02550773, 0.0328173 , 0.02887736, 0.14533653, 0.02409933,
       0.03024066, 0.01445311, 0.04350997, 0.04280766, 0.02020658,
       0.02575523, 0.03984161, 0.0471417 , 0.01894485, 0.02851368,
       0.02005321, 0.03796182, 0.02418424, 0.08893508, 0.01769097,
       0.01300851, 0.03880661, 0.02692984, 0.04639288])

In [17]:
tmp = torch.load('../../model/v12/stage1-fold0-seed4/best.pth', map_location='cpu')
sigmoid(tmp['val_preds']).mean(axis=0)

array([0.02747452, 0.04221486, 0.02012031, 0.10325328, 0.02895558,
       0.02939461, 0.01261754, 0.03390609, 0.04031887, 0.02480255,
       0.02672071, 0.04011226, 0.06316055, 0.03911912, 0.02643656,
       0.03016641, 0.03283299, 0.0200389 , 0.09705085, 0.01645752,
       0.00842396, 0.03952034, 0.02865285, 0.05285057])

In [18]:
tmp = torch.load('../../model/v12/stage2-fold0-seed7/best.pth', map_location='cpu')
sigmoid(tmp['val_preds']).mean(axis=0)

array([0.02781067, 0.04668172, 0.02703697, 0.10870536, 0.02981778,
       0.03547848, 0.01357744, 0.03450568, 0.03964604, 0.02652382,
       0.03314757, 0.04319209, 0.0555724 , 0.04192604, 0.03169692,
       0.03394956, 0.03976284, 0.02233029, 0.09820811, 0.01922553,
       0.01234154, 0.04294818, 0.02938584, 0.04969589])

In [19]:
tmp = torch.load('../../model/v12/stage3-fold0-seed10/best.pth', map_location='cpu')
sigmoid(tmp['val_preds']).mean(axis=0)

array([0.03006334, 0.0457178 , 0.03074395, 0.10823   , 0.0318668 ,
       0.03610059, 0.01459556, 0.03838072, 0.04177073, 0.03069071,
       0.03314655, 0.05082011, 0.05740452, 0.04450365, 0.02984581,
       0.04053078, 0.04416258, 0.02290087, 0.09969498, 0.02020152,
       0.01447978, 0.04352839, 0.03205069, 0.048899  ])

In [6]:
train_tp.groupby('species_id').size()/train_fp.groupby('species_id').size()

species_id
0     0.166667
1     0.167785
2     0.123333
3     0.166667
4     0.166667
5     0.166667
6     0.166667
7     0.166667
8     0.166667
9     0.113333
10    0.167785
11    0.166667
12    0.166667
13    0.166667
14    0.166667
15    0.166667
16    0.166667
17    0.113333
18    0.166667
19    0.143813
20    0.133333
21    0.166667
22    0.153846
23    0.166667
dtype: float64

In [10]:
def transform(x, offset):
    return frommelscale(tomelscale(x)+offset).astype(int)
offset = 100

df = pd.concat((train_tp, train_fp), axis=0)
for i in range(24):
    cur = df.loc[df['species_id']==i]
    tmp = transform(cur['f_min'], -offset).astype(str) + '-' + transform(cur['f_max'], offset).astype(str)
    print(i, tmp.unique())

0 ['5345-9080' '5345-8260']
1 ['3457-6211']
2 ['369-3343']
3 ['884-3138']
4 ['2085-4675' '2256-4470']
5 ['4315-11436' '4573-12358']
6 ['455-4880' '455-3548']
7 ['4230-12563']
8 ['3372-6109']
9 ['712-6314' '970-5802']
10 ['807-11924']
11 ['1595-6277']
12 ['455-3650']
13 ['26-986' '26-1601']
14 ['2256-6109' '3372-7133' '2514-4880']
15 ['26-1294']
16 ['2771-5084']
17 ['1280-5053' '1227-4367' '1141-4367' '1141-8158']
18 ['2857-5597']
19 ['197-3138']
20 ['1141-6724' '2085-6314']
21 ['2943-4572']
22 ['9720-15022']
23 ['6561-12395' '5852-12771']


In [8]:
import pandas as pd
from utils import frommelscale, tomelscale
offset = 100
def transform(x, offset):
    return frommelscale(tomelscale(x)+offset).astype(int)
train_tp = pd.read_csv('../../input/kaggle/train_tp.csv', index_col='recording_id')
train_fp = pd.read_csv('../../input/kaggle/train_fp.csv', index_col='recording_id')
df = pd.concat((train_tp, train_fp), axis=0)
res = pd.DataFrame(index=range(24), columns=['fmin', 'fmax'])
for i in range(24):
    cur = df.loc[df['species_id']==i]
    res.loc[i, 'fmin'] = cur['f_min'].min()
    res.loc[i, 'fmax'] = cur['f_max'].max()

In [13]:
res.to_feather('fminfmax_by_sid.ft')

In [12]:
res

,f_min,f_max
0,5906.2500,8250.0000
1,3843.7500,5625.0000
2,468.7500,3000.0000
3,1031.2500,2812.5000
4,2343.7500,4218.7500
5,4781.2500,11250.0000
6,562.5000,4406.2500
7,4687.5000,11437.5000
8,3750.0000,5531.2500
9,843.7500,5718.7500


In [15]:
train_df = pd.DataFrame(0, index=recording_ids, columns=range(24))
for i in range(24):
    cur = train_tp[train_tp['species_id']==i]
    train_df.loc[cur.index, i] = 1
train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
73c482b64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
afaa8d38f,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
074cf066a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
de0c8a7f7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
e568d45b5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# LOSS ERROR

In [7]:
data, logits, loss, grad_norm = joblib.load('error.jl')